# **Python, R & CUDA: How to handle them on conda environments**
## **Python Environment (For verify a specific Python version)**
In this case, we want to use `Python 3.8.10` to run FADCIL. Note that we have created the directory `environment/PyVersionTest` for better organization and to isolate the following `.yml` file from other environments. Let's take a look:
```bash
(base) [u6059911@kingspeak2:~]$ cd ~/environments/PyVersionTest && ls
PyVersion.yml
(base) [u6059911@kingspeak2:PyVersionTest] cat PyVersion.yml
name: Env_Py_3_8
channels:
  - defaults
  - conda-forge
dependencies:
  - python=3.8.10
  - pip
  - numpy
  - pandas
  - matplotlib
  - scikit-learn
  - jupyterlab
  ```
Where `PyVersion.yml` defines the dependencies we want in the `Env_py_3_8` environment. Furthermore, once we activate that environment, we have:
  ```bash
(base) [u6059911@kingspeak2:env_test]$ conda activate Env_Py_3_8
(Env_Py_3_8) [u6059911@kingspeak2:env_test]$ python3 --version
Python 3.8.10
```
Where we have check that, in this environment, we are working with `Python3.8.10`. If we deactivate the environment (i.e, return  to the `base` environment), we have:
```bash
(base) [u6059911@kingspeak2:~]$ python3 --version
Python 3.12.4
```
Therefore, note that conda environments work in isolation.
## **R environment (For verify that R also works under the same package download scheme on conda)**
Let's verify that we can download R dependencies using `.yml` files and create R conda environments. Note the path `~/environments/Rtest`:
```bash
(base) [u6059911@kingspeak2:~]$ cd ~/environments/Rtest && ls
Renv.yml CheckDependencies.R
```
Now let's see what these files contain on:
```bash
(base) [u6059911@kingspeak2:Rtest] cat Renv.yml
name: R_env
channels:
  - defaults
  - conda-forge
  - r
dependencies:
  - r-base=4.2.0 
  - r-dplyr
  - r-tidyverse
  - openjdk=8
(base) [u6059911@kingspeak2:Rtest] cat CheckDependencies.R
packages <- c("dplyr", "tidyverse", "h2o")

for (pkg in packages) {
  if (require(pkg, character.only = TRUE)) {
    cat(sprintf("Paquete %s está instalado.\n", pkg))
  } else {
    cat(sprintf("Paquete %s no está instalado.\n", pkg))
  }
}
```
Where `Renv.yml` defines the `R_env` environment (with R dependencies), and `CheckDependencies.R` is a script that checks all dependencies were installed correctly and are ready to be used. Now, we need to activate the environment:
```bash
(base) [u6059911@kingspeak2:Rtest]$ conda activate R_env
```
And then run the checking script:
```bash
(R_env) [u6059911@kingspeak2:Rtest]$ Rscript CheckDependencies.R
Loading required package: dplyr

Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Paquete dplyr está instalado.
Loading required package: tidyverse
── Attaching packages ────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──
✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ stringr 1.4.0
✔ tidyr   1.2.0     ✔ forcats 0.5.1
✔ readr   2.1.2     
── Conflicts ───────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Paquete tidyverse está instalado.
Loading required package: h2o

----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit https://docs.h2o.ai

----------------------------------------------------------------------


Attaching package: ‘h2o’

The following objects are masked from ‘package:stats’:

    cor, sd, var

The following objects are masked from ‘package:base’:

    &&, %*%, %in%, ||, apply, as.factor, as.numeric, colnames,
    colnames<-, ifelse, is.character, is.factor, is.numeric, log,
    log10, log1p, log2, round, signif, trunc

Paquete h2o está instalado.
Warning message:
package ‘h2o’ was built under R version 4.2.3 

```
Which indicates that the R dependencies can also be used using the conda environment scheme.
## **CUDA environment (For set up a fixed CUDA version on an isolate environment)**
Now we want to know how to install a specific CUDA version in a coda environment. We hope that the CUDA version we install will be available only in that specific environment for better isolation. Let's take a look:
```bash
(base) [u6059911@kingspeak2:~]$ cd ~/environments/CUDAtest && ls
CuVersion.yml
(base) [u6059911@kingspeak2:Rtest] cat CuVersion.yml
name: CUDA_env
channels:
  - defaults
  - nvidia
dependencies:
  - cudatoolkit=10.1
```
Now once we activate the environment:
```bash
(base) [u6059911@kingspeak2:Rtest]$ conda activate CUDA_env
```
We need to take the following step to ensure that the CUDA version module is automatically loaded each time we activate the environment:
```bash
(CUDA_env) [u6059911@kingspeak2:Rtest]$ mkdir -p $CONDA_PREFIX/etc/conda/activate.d
(CUDA_env) [u6059911@kingspeak2:Rtest]$ echo "module load cuda/10.1" > $CONDA_PREFIX/etc/conda/activate.d/load_cuda.sh
```
Now, we can test the result leaving the environment and joining again:
```bash
(CUDA_env) [u6059911@kingspeak2:Rtest]$ conda deactivate CUDA_env
(base) [u6059911@kingspeak1:cuda_test]$ conda activate CUDA_env
```
Let's see the CUDA version:
```bash
(CUDA_env) [u6059911@kingspeak1:cuda_test]$ nvcc --version
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Wed_Apr_24_19:10:27_PDT_2019
Cuda compilation tools, release 10.1, V10.1.168
```
So the CUDAV10.1 was installed successfully. We can also verify that `nvcc` will only work inside `CUDA_env`. Note that:
```bash
(CUDA_env) [u6059911@kingspeak2:Rtest]$ echo $CONDA_PREFIX
/uufs/chpc.utah.edu/common/home/u6059911/software/pkg/miniconda3/envs/CUDA_env
```
Returns the `CUDA_env` path, so we can find `load_cuda.sh` under this way: 
```bash
(CUDA_env) [u6059911@kingspeak2:Rtest]$ ls $CONDA_PREFIX/etc/conda/activate.d/
load_cuda.sh
```
But, for example, if we would have been on another environment, like `Env_Py_3_8`:
```bash
(Env_Py_3_8) [u6059911@kingspeak1:cuda_test]$ ls $CONDA_PREFIX/etc/conda/activate.d/
libblas_mkl_activate.sh
```
That means, we would'nt have found the `load_cuda.sh`, which means thant `nvcc` would'nt be available in `Env_Py_3_8` (and, therefore, CUDA would not be available).